# Ngày 3: Transformations & Actions

#### Transformation & action là hai operation trên RDD. Ta cùng tìm hiểu thôi nào :))

### 1. Lazy Evaluation (lazy là lười biếng)

Việc xử lý dữ liệu, nhìn rộng ra, chính là việc biến đổi từ tập dữ liệu này sang tập dữ liêu khác, hay với Spark, là biến đổi từ RDD này sang RDD khác.

**Khái niệm**: Spark không thực hiện tính toán ngay lập tức khi bạn gọi các transformations. Thay vào đó, Spark xây dựng một "DAG" (Directed Acyclic Graph) và chỉ thực hiện tính toán khi có một action được gọi.

- Lợi ích: Tối ưu hóa các bước tính toán
- Giảm thiểu số lượng bước thừa

**Example**

Ví dụ: Ở đây ta có 1 tập dữ liệu với 1 tỉ record chứa dữ liệu về các nước... Ta sẽ thực hiện các
transformation trên tập này như hình bên dưới.

![image](image_1/lazy.png)

**Giải thích**: Ở hình trên ta đang thực hiện 3 transformation là:
- RDD1: Load to memory
- RDD2: group by()
- RDD3: Filter from RDD2
- Action: show()

<p style = "color: green;">Vậy Spark lazy như thế nào ? </p>

Ở đây ta có thể thấy mặc dù đã tạo ra 3 RDD nhưng thực chất không có RDD nào được thực thi cả. Spark cứ
đợi cho đến khi nào có một action được gọi thì spark mới bắt đầu thực hiện phân tích các transfomation và
đưa ra strategy tối ưu nhất.

--> Chính việc đợi và đợi cho đến khi có một action được gọi chính là cơ chế LAZY

<p style = "color: green;">Vậy tại sao Spark lại phải lazy ? </p>

![whylazy](image_1/notlazy.png)


**Giải thích:** Bạn so sánh khi lazy và không lazy xem. 
- Nếu không lazy thì cứ làm việc tuần tự như thế,
nó sẽ load 1 tỷ recore lên memory, tốn rất nhiều bộ nhớ.
- Còn lazy, nó làm biếng nhưng thông minh hơn, nó xem xét thấy bước
filter làm trước thì dữ liệu load lên memory chỉ 5 record,
tiết khiệm rất nhiều dung lượng bộ nhớ.

**Tóm lại:** Việc lazy là để cho spark có khả năng tính toán, lập kế hoạch và đưa ra được cái plan
tối ưu nhất.

### 2. Narrow vs. Wide transformations

Transformations có hai loại và narrow và wide. Vậy hai loại này khác nhau như thế nào ?

![shuffle](image_1/check.png)

Sự khác biệt lớn nhất nằm ở việc có hay không có **shuffle**


- Với narrow transformation: map, filter --> Không có shuffle
- Với wide transformation: reduceByKey, sortByKey --> Có shuffle

**Vậy shuffle ở đây là gì ?**

Shuffle là hiện tượng xáo trộn dữ liệu giữa các node trong cluster để thu được kết quả đúng yêu cầu.

![narrow and wide](image_1/compare.png)

- Giải thích một chút về dữ liệu trong ảnh:
  + Bối cảnh: Giả sử có một file dữ liệu lớn với 50k record. Yêu cầu tìm số lượng order của mỗi loại status(Close, complete, pending.....)
  + Sử dụng hệ thống Spark, dữ liệu trong hdfs được chia thành 5 phần do 5 node trong cluster quản lí. Mỗi node quản lí 10k record.
  + Để giải quyết tìm số lượng order của các status, ta thực hiện quy trình như trong ảnh: Load to Memory --> Map --> reduceByKey --> Collect
  + Ở đây ta nhìn vào **map**: Mặc dù dữ liệu chuyển đổi từ string sang dictionary tuy nhiên bản chất nó vẫn vậy và nằm trong 1 node
  + Ở đây ta nhìn vào **ReduceByKey**: Để tổng hợp số lượng Order của mỗi status, bắt buộc phải đưa dữ liệu về một chỗ, ở đây cụ thể là một node để tính. Ví dụ như trên hình trên, ta đã di chuyển dữ liệu Close về lại node đầu tiên để tổng hợp và tính.
  + Việc đưa dữ liệu di chuyển rồi tổng hợp về một node như vậy gọi là Shuffle
- Chỉ có **reduceByKey** mới shuffle nên gọi nó là **Wide**, còn **map** bản chất dữ liệu vẫn nằm tại một node nên gọi là **narrow**
- Vì việc di chuyển dữ liệu giữa các node như vậy tốn rất nhiều dung lượng bộ nhớ làm cho chương trình thực thi bị chậm lại, nên trong các phép biến đổi transformation, ta ưu tiên đưa các narrow transformation lên trên đầu và đưa các wide transformation thực hiện ở các bước cuối để tối ưu hóa hiệu năng.

### 3. Các transformations phổ biến

- **map(func)**: Áp dụng func lên tưng phần tử
- **filter(func)**: Giữ lại các phần tử thỏa mãn func
- **flatMap(func)**: Áp dụng func rồi trả về từng phần tử của output
- **reduceByKey(func)**: Gom nhóm và áp dụng func theo từng khóa.
- **groupByKey()**: Gom nhóm dữ liệu theo khóa.
- **join(otherRDD)**: Ghép 2 RDD dựa trên khóa chung.
- **distinct()**: Loại bỏ phần tử trùng lặp.
- **sample(withReplacement, fraction)**: Lấy mẫu từ dữ liệu.

![famous transfomation](image_1/transfomation.png)


<p style = 'color: green;'>Đặc điểm chung của các hàm trên:</p>

- Đều sử dụng kèm theo một **function** bên trong, ngoại trừ *distinct*
- Khi sử dụng những transformation này, ta quan tâm đến số lượng **input** và **output**

<p style = 'color: green;'>Đếm dòng:</p>

![famous transformation](image_1/transfomation_detail.png)

- Map(func): Input là 1000 dòng thì Output là 1000 dòng. Khi làm bài cần số lượng output bằng số lượng input,
ta liên tưởng đến hàm Map
- Reduce(func): Chỉ trả về đúng 1 dòng thỏa điều hiện hàm.
- ReduceByKey(func): Số lượng Output bằng số lượng KEY
- Filter(func): Với spark, nên ưu tiên sử dụng transfomation filter này dùng trên cùng. Hàm lọc ra các giá trị thỏa mãn điều kiện. Số lượng OUTPUT < INPUT
- SortBy(func): Sắp xếp giá trị, có thể dùng để tìm các giá trị lớn nhất, nhỏ nhất. Số lượng OUTPUT = INPUT
- Distinct(): Cho ra các giá trị duy nhất, số lượng OUTPUT < INPUT

### 4. Các Action phổ biến

- **collect()** : Trả về tất cả dữ liệu về drive (chỉ dùng cho dữ liệu nhỏ)
- **count()**: Đếm số lượng phần tử
- **take(n)**: Lấy n phàn tử đầu
- **saveAsTextFile(path)**: Lưu kết quả vào thử mục theo dạng file văn bản

## Bài tập thực hành

**Bài 1**: Tìm 10 từ xuất hiện nhiều nhất trong một tập tin văn bản lớn

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .appName("WordCount") \
    .getOrCreate()

24/11/06 11:10:59 WARN Utils: Your hostname, ubunchuu-Test resolves to a loopback address: 127.0.1.1; using 10.131.6.208 instead (on interface wlo1)
24/11/06 11:10:59 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/06 11:11:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


24/11/06 11:11:12 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [4]:
sc = spark.sparkContext

In [5]:
rdd = sc.textFile("material_1/vanban.txt")

In [6]:
# Dem word count trong van ban
word_count = rdd.flatMap(lambda x: x.split()) \
                .map(lambda word : (word,1)) \
                .reduceByKey(lambda x, y: x+y)

In [7]:
word_count.collect()

[('Với', 1),
 ('nhiều', 2),
 ('ra', 5),
 ('trong', 4),
 ('gia', 1),
 ('đình', 1),
 ('nhập', 6),
 ('Florida', 2),
 ('đe', 1),
 ('dọa', 1),
 ('xuất', 6),
 ('loạt"', 1),
 ('không', 1),
 ('lo', 2),
 ('bằng', 1),
 ('đề', 3),
 ('kinh', 1),
 ('Bước', 1),
 ('khỏi', 2),
 ('phòng', 1),
 ('bỏ', 1),
 ('Hialeah,', 1),
 ('thị', 1),
 ('với', 5),
 ('Florida,', 1),
 ('24', 1),
 ('tiết', 1),
 ('lộ', 1),
 ('đã', 3),
 ('cựu', 1),
 ('tổng', 3),
 ('thống', 2),
 ('thứ', 1),
 ('Mỹ,', 2),
 ('Colorado', 1),
 ('Nam', 1),
 ('Mỹ.', 1),
 ('được', 1),
 ('đánh', 1),
 ('một', 1),
 ('những', 2),
 ('chủ', 2),
 ('chốt', 1),
 ('năm', 2),
 ('nay.', 1),
 ('Trump', 3),
 ('lần', 1),
 ('nếu', 2),
 ('phát', 1),
 ('trục', 5),
 ('lớn', 1),
 ('nhất', 1),
 ('lịch', 1),
 ('Mỹ",', 1),
 ('thậm', 1),
 ('định', 1),
 ('trao', 1),
 ('quốc', 1),
 ('Mỹ', 2),
 ('trên', 1),
 ('lãnh', 1),
 ('thổ', 1),
 ('visa', 1),
 ('du', 1),
 ('chuyển', 1),
 ('trước', 1),
 ('đang', 1),
 ('việc', 1),
 ('Miami,', 1),
 ('thông', 1),
 ('mà', 1),
 ('đưa', 1),
 ('

In [8]:
# Sap xep giam dan va loc ra 10 gia tri dau tien
top_10_word = word_count.sortBy(lambda x: x[1], ascending=False)

top_10_word.take(10)

[('người', 7),
 ('nhập', 6),
 ('xuất', 6),
 ('cư', 6),
 ('ra', 5),
 ('với', 5),
 ('trục', 5),
 ('ông', 5),
 ('là', 5),
 ('trong', 4)]

**Bài 2:** Tính tổng doanh thu theo từng danh mục sản phẩm từ một RDD chứa dữ liệu bán hàng.

In [55]:
# Load to memory
sales_rdd = sc.textFile("material_1/online_sales_data.csv")

In [56]:
# Xem so luong dong
sales_rdd.count()

240

In [57]:
# Xem 10 phan tu dau tien
sales_rdd.take(10)

['10001,2024-01-01,Electronics,iPhone 14 Pro,2,999.99,1999.98,North America,Credit Card',
 '10002,2024-01-02,Home Appliances,Dyson V11 Vacuum,1,499.99,499.99,Europe,PayPal',
 "10003,2024-01-03,Clothing,Levi's 501 Jeans,3,69.99,209.97,Asia,Debit Card",
 '10004,2024-01-04,Books,The Da Vinci Code,4,15.99,63.96,North America,Credit Card',
 '10005,2024-01-05,Beauty Products,Neutrogena Skincare Set,1,89.99,89.99,Europe,PayPal',
 '10006,2024-01-06,Sports,Wilson Evolution Basketball,5,29.99,149.95,Asia,Credit Card',
 '10007,2024-01-07,Electronics,MacBook Pro 16-inch,1,2499.99,2499.99,North America,Credit Card',
 '10008,2024-01-08,Home Appliances,Blueair Classic 480i,2,599.99,1199.98,Europe,PayPal',
 '10009,2024-01-09,Clothing,Nike Air Force 1,6,89.99,539.94,Asia,Debit Card',
 '10010,2024-01-10,Books,Dune by Frank Herbert,2,25.99,51.98,North America,Credit Card']

In [58]:
# # Lấy ra category và total_price

# Định nghia ham xu li tung line de lay ra category va total_price
def safe_parse(line):
    parts = line.split(",")
    # Kiểm tra xem có đủ phần tử
    if len(parts) > 6:
        try:
            # Lấy category (cột thứ 3) và giá trị doanh thu (cột thứ 7)
            category = parts[2]
            price = float(parts[6])  # Chuyển đổi thành float
            return (category, price)
        except ValueError:
            # Bỏ qua dòng nếu không thể chuyển đổi sang float
            print(f"Skipping line due to ValueError: {line}")
            return None
    else:
        # Bỏ qua dòng nếu không đủ phần tử
        print(f"Skipping line due to insufficient data: {line}")
        return None



In [59]:
# Áp dụng hàm safe_parse và loại bỏ các dòng None

type_map_sales_rdd = sales_rdd.map(safe_parse).filter(lambda x: x is not None)

In [60]:
# Take 10 dong
type_map_sales_rdd.take(10)

[('Electronics', 1999.98),
 ('Home Appliances', 499.99),
 ('Clothing', 209.97),
 ('Books', 63.96),
 ('Beauty Products', 89.99),
 ('Sports', 149.95),
 ('Electronics', 2499.99),
 ('Home Appliances', 1199.98),
 ('Clothing', 539.94),
 ('Books', 51.98)]

In [61]:
# Tinh tong doanh thu theo tung category

revenue_rdd = type_map_sales_rdd.reduceByKey(lambda x, y: x+y)

In [62]:
revenue_rdd.take(10)

Skipping line due to ValueError: 10016,2024-01-16,Books,"Salt, Fat, Acid, Heat by Samin Nosrat",3,35.99,107.97,North America,Credit Card


[('Electronics', 34982.40999999999),
 ('Books', 1753.9600000000005),
 ('Home Appliances', 18646.159999999996),
 ('Clothing', 8128.9299999999985),
 ('Beauty Products', 2621.9),
 ('Sports', 14326.519999999999)]

**Bài 3:** Ghép hai RDD dựa trên một khóa chung

In [63]:
# Giả sử rdd1 và rdd2 chứa dữ liệu với ID chung
rdd1 = sc.parallelize([(1, "Alice"), (2, "Bob"), (3, "Carol")])
rdd2 = sc.parallelize([(1, 25), (2, 30), (4, 40)])

# Ghép hai RDD dựa trên ID
joined_rdd = rdd1.join(rdd2)

In [64]:
joined_rdd.collect()

[(1, ('Alice', 25)), (2, ('Bob', 30))]

Hai rdd trên chung khóa 1 và 2 nên dùng hai khóa đó ghép lại